In [2]:
import numpy as np

In [79]:
game_state ={
    'round':1,
    'step':1,
    'field':np.zeros((field_size,field_size)),
    'bombs':[((2,3),1),((3,3),3)],
    'explosion_map':np.zeros((field_size,field_size)),
    'coins':[(1,1),(2,1)],
    'self':('PyBomb',3,True,(1,3)), #name,score, Bomb possible, coordinates
    'others':[('enemy_0',3,True,(2,3)),('enemy_1',3,True,(3,3)),('enemy_2',3,False,(1,3))]
}

In [128]:
def reshape_game_state(game_state,field_size, vector=True):
    '''
    INPUT: game_state dict, size of field, vector = True
    OUTPUT: all relevant game info, in the shape of [field_size,field_size] or as stacked vector of length 5*field_size**2
    
    Function reshapes all information in dictionary 
    returns info in the form of maps or as unravelled stacked vector
    '''
    #extract all data from coin, bomb and players to maps
    coin_map = np.zeros((field_size,field_size))
    for coin_coord in game_state['coins']:
        coin_map[coin_coord]=1
        
    bomb_map = np.zeros((field_size,field_size))
    for bomb_coord,bomb_time in game_state['bombs']:
        bomb_map[bomb_coord]=bomb_time
        
    player_map = np.zeros((field_size,field_size))
    for name,score,bomb,coord in game_state['others']:
        if bomb:
            player_map[coord]= -3
        else:
            player_map[coord]=-1
    name,score,bomb,coord = game_state['self']
    if bomb:
        player_map[coord]= 3
    else:
        player_map[coord]= 1
        
    #join all maps
    game_info = np.stack([game_state['field'],game_state['explosion_map'],coin_map,bomb_map,player_map])
    if vector:
        game_info = game_info.reshape(-1)
    
    return game_info

In [130]:
reshape_game_state(game_state,5, vector=False)

array([[[ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  3.,  0.],
        [ 0.,  0.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  3.,  0.],
        [ 0.,  0.,  0., -3.,  0.],
        [ 0.,  0.,  0., -3.,  0.],
        [ 0.,  0.,  0.,  0.,  0.]]])

In [131]:
import tensorflow as tf
from tensorflow.keras.layers import (Input,Dense,Lambda)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam



In [133]:

def build_q_network(n_actions, learning_rate=0.00001, input_shape=(125), history_length=4):
    """Builds a DQN as a Keras model
    
    Arguments:
        n_actions: Number of possible action the agent can take
        learning_rate: Learning rate
        input_shape: Shape of the preprocessed frame the model sees
        history_length: Number of historical frames the agent can see
        
    Returns:
        A compiled Keras model
    """
    model_input = Input(shape=(input_shape, history_length))

    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    x = Dense(32, activation='relu')(x)
    x = Dense(16, activation='relu')(x)
    
    q_vals = Dense(n_actions,activation='softmax')(x)

    # Build model
    model = Model(model_input, q_vals)
    model.compile(Adam(learning_rate), loss=tf.keras.losses.Huber())

    return model

In [135]:

def build_q_network(n_actions, learning_rate=0.00001, input_shape=(125), history_length=4):
    """Builds a DQN as a Keras model
    
    Arguments:
        n_actions: Number of possible action the agent can take
        learning_rate: Learning rate
        input_shape: Shape of the preprocessed frame the model sees
        history_length: Number of historical frames the agent can see
        
    Returns:
        A compiled Keras model
    """
    model_input = Input(shape=(input_shape, history_length))

    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    x = Dense(32, activation='relu')(x)
    x = Dense(16, activation='relu')(x)
    
    value = Dense(1)(x)
    
    action = Dense(n_actions,activation='softmax')(x)

    Q = value + tf.subtract(actions, tf.reduce_mean(actions, axis=1, keepdims=True))
    # Build model
    model = Model(model_input, q_vals)
    model.compile(Adam(learning_rate), loss=tf.keras.losses.Huber())

    return model